# Emotion Recognition from Speech

Using neural networks, audio data from actors were used to train a model to associate emotions with speech.

<hr></hr>

## Module, Functions, and Definitions

In [30]:
import librosa  #  Analyze audio files
import soundfile  #  Read sound files
import os, glob, joblib
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier  #  Multi-Layer Perceptron classifier (Artificial Neural Network)
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import GridSearchCV

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [3]:
# Function used to cycle through audio files and extract features to use to train model

def extract_feature(file_name):
    with soundfile.SoundFile(file_name) as sf:
        X = sf.read(dtype="float32")
        sample_rate = sf.samplerate
        
        result = np.array([])
        
# Mel-frequency cepstral coefficients - More accuratly represents human speech
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)

# Chromagram from a waveform - Returns normalized energy values for 12 distinct semitones
        stft = np.abs(librosa.stft(X))
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
          
# Mel-scaled spectrogram - Frequency histogram based on percieved frequencies
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        
    return np.hstack((mfccs, chroma, mel))

In [4]:
# Dictionary of emotions from dataset

emotions = {
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

## Data Pre-processing

The audio files were obtained from the Ryerson Audio-Visiual Database of Emotional Speech and Song (RAVDESS), which consists of 24 actors saying the same short phrase in 8 different emotions.

Data was acquired here:  https://zenodo.org/record/1188976/files/Audio_Speech_Actors_01-24.zip?download=1

In [5]:
# Read through all audio files, extract features, and split into testing and training groups

X,y = [], []

for file in glob.glob("data/Actor_*/*.wav"):
    name = os.path.basename(file)
# Emotions are indicated with a number in the file name corresponding to that defined in our dictionary.
    emotion = emotions[name.split("-")[2]]
    feature = extract_feature(file)
    
    X.append(feature)
    y.append(emotion)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, stratify=y, random_state=42)

print(f'Number of training data: {X_train.shape[0]}\nNumber of testing data: {X_test.shape[0]}')

Number of training data: 1076
Number of testing data: 359


In [9]:
# Normalize testing and training audio feature data

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Classify emotions with number representation

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [11]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [13]:
print(f'Shape of data: {X_train_scaled[0].shape[0], y_train_categorical[0].shape[0]}')

Shape of data: (180, 8)


## Model Training

Two methods were used to train a model to fit the data: a TensorFlow neural network; and a Scikit-Learn neural network.

In [14]:
model = Sequential()
model.add(Dense(units=300, activation='relu', input_dim=180))
model.add(Dense(units=300, activation='relu'))
model.add(Dense(units=300, activation='relu'))
model.add(Dense(units=300, activation='relu'))
model.add(Dense(units=300, activation='relu'))
model.add(Dense(units=8, activation='softmax'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               54300     
_________________________________________________________________
dense_1 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_2 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_3 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_4 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 2408      
Total params: 417,908
Trainable params: 417,908
Non-trainable params: 0
__________________________________________________

In [17]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=300,
    shuffle=True,
    verbose=1
)

Train on 1076 samples
Epoch 1/300
1076/1076 [==============================] - 1s 529us/sample - loss: 1.9749 - accuracy: 0.2017
Epoch 2/300
1076/1076 [==============================] - 0s 116us/sample - loss: 1.7628 - accuracy: 0.2937
Epoch 3/300
1076/1076 [==============================] - 0s 114us/sample - loss: 1.6487 - accuracy: 0.3299
Epoch 4/300
1076/1076 [==============================] - 0s 117us/sample - loss: 1.5574 - accuracy: 0.3866
Epoch 5/300
1076/1076 [==============================] - 0s 113us/sample - loss: 1.5232 - accuracy: 0.4071
Epoch 6/300
1076/1076 [==============================] - 0s 117us/sample - loss: 1.4759 - accuracy: 0.4210
Epoch 7/300
1076/1076 [==============================] - 0s 190us/sample - loss: 1.3660 - accuracy: 0.4833
Epoch 8/300
1076/1076 [==============================] - 0s 118us/sample - loss: 1.3344 - accuracy: 0.4954
Epoch 9/300
1076/1076 [==============================] - 0s 115us/sample - loss: 1.2562 - accuracy: 0.5149
Epoch 10/300
10

1076/1076 [==============================] - 0s 118us/sample - loss: 0.0837 - accuracy: 0.9758
Epoch 77/300
1076/1076 [==============================] - 0s 118us/sample - loss: 0.1264 - accuracy: 0.9600
Epoch 78/300
1076/1076 [==============================] - 0s 119us/sample - loss: 0.1353 - accuracy: 0.9489
Epoch 79/300
1076/1076 [==============================] - 0s 120us/sample - loss: 0.1938 - accuracy: 0.9322
Epoch 80/300
1076/1076 [==============================] - 0s 109us/sample - loss: 0.1232 - accuracy: 0.9545
Epoch 81/300
1076/1076 [==============================] - 0s 116us/sample - loss: 0.1192 - accuracy: 0.9591
Epoch 82/300
1076/1076 [==============================] - 0s 115us/sample - loss: 0.2298 - accuracy: 0.9182
Epoch 83/300
1076/1076 [==============================] - 0s 118us/sample - loss: 0.2177 - accuracy: 0.9219
Epoch 84/300
1076/1076 [==============================] - 0s 120us/sample - loss: 0.1461 - accuracy: 0.9442
Epoch 85/300
1076/1076 [=================

1076/1076 [==============================] - 0s 179us/sample - loss: 2.1169e-04 - accuracy: 1.0000
Epoch 151/300
1076/1076 [==============================] - 0s 179us/sample - loss: 2.0211e-04 - accuracy: 1.0000
Epoch 152/300
1076/1076 [==============================] - 0s 146us/sample - loss: 1.9311e-04 - accuracy: 1.0000
Epoch 153/300
1076/1076 [==============================] - 0s 157us/sample - loss: 1.8449e-04 - accuracy: 1.0000
Epoch 154/300
1076/1076 [==============================] - 0s 164us/sample - loss: 1.7776e-04 - accuracy: 1.0000
Epoch 155/300
1076/1076 [==============================] - 0s 153us/sample - loss: 1.7328e-04 - accuracy: 1.0000
Epoch 156/300
1076/1076 [==============================] - 0s 146us/sample - loss: 1.6428e-04 - accuracy: 1.0000
Epoch 157/300
1076/1076 [==============================] - 0s 146us/sample - loss: 1.5724e-04 - accuracy: 1.0000
Epoch 158/300
1076/1076 [==============================] - 0s 146us/sample - loss: 1.5266e-04 - accuracy: 1.00

1076/1076 [==============================] - 0s 190us/sample - loss: 2.4404e-05 - accuracy: 1.0000 - loss: 2.0654e-05 - accuracy: 1.
Epoch 223/300
1076/1076 [==============================] - 0s 168us/sample - loss: 2.3949e-05 - accuracy: 1.0000
Epoch 224/300
1076/1076 [==============================] - 0s 174us/sample - loss: 2.3319e-05 - accuracy: 1.0000
Epoch 225/300
1076/1076 [==============================] - 0s 155us/sample - loss: 2.2789e-05 - accuracy: 1.0000
Epoch 226/300
1076/1076 [==============================] - 0s 163us/sample - loss: 2.2189e-05 - accuracy: 1.0000
Epoch 227/300
1076/1076 [==============================] - 0s 150us/sample - loss: 2.1624e-05 - accuracy: 1.0000
Epoch 228/300
1076/1076 [==============================] - 0s 151us/sample - loss: 2.1120e-05 - accuracy: 1.0000
Epoch 229/300
1076/1076 [==============================] - 0s 159us/sample - loss: 2.0849e-05 - accuracy: 1.0000
Epoch 230/300
1076/1076 [==============================] - 0s 160us/sample -

1076/1076 [==============================] - 0s 176us/sample - loss: 4.8056e-06 - accuracy: 1.0000
Epoch 295/300
1076/1076 [==============================] - 0s 171us/sample - loss: 4.6878e-06 - accuracy: 1.0000
Epoch 296/300
1076/1076 [==============================] - 0s 150us/sample - loss: 4.5672e-06 - accuracy: 1.0000
Epoch 297/300
1076/1076 [==============================] - 0s 155us/sample - loss: 4.5640e-06 - accuracy: 1.0000
Epoch 298/300
1076/1076 [==============================] - 0s 151us/sample - loss: 4.3931e-06 - accuracy: 1.0000
Epoch 299/300
1076/1076 [==============================] - 0s 150us/sample - loss: 4.3101e-06 - accuracy: 1.0000
Epoch 300/300
1076/1076 [==============================] - 0s 146us/sample - loss: 4.2360e-06 - accuracy: 1.0000


In [33]:
# Test accuracy of model
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)

print(f"TensorFlow Results - Loss: {model_loss}, Accuracy: {model_accuracy}")

359/359 - 0s - loss: 3.1276 - accuracy: 0.6908
TensorFlow Results - Loss: 3.1276437306470526, Accuracy: 0.6908078193664551


In [34]:
# Save model
model.save("tf_model.h5")

In [20]:
mlpModel = MLPClassifier(
    alpha = 0.01,
    batch_size = 256,
    epsilon = 1e-08,
    hidden_layer_sizes = (300,),
    learning_rate = 'adaptive',
    max_iter = 1000,
    #shuffle = False,
    #n_iter_no_change = 20,
    #learning_rate_init = 0.0001,
    #beta_1 = 0.1,
    #beta_2 = 0.9,
    warm_start = True
)

In [21]:
mlpModel.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=1000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=True)

In [29]:
# Test accuracy of model
y_pred = mlpModel.predict(X_test)

print(f'MLP Results - Accuracy: {accuracy_score(y_true=y_test, y_pred=y_pred)}')

MLP Results - Accuracy: 0.4986072423398329


In [32]:
# Save model
joblib.dump(mlpModel, "mlp_model.sav")

['mlp_model.sav']